In [40]:
#!/usr/bin/env python3
from glob import glob
import os.path
import os
import shutil
import yaml
from jinja2 import Template

build_path = "./build"
plan_path = "./plan.yaml"
meals_path = "./meals"


def get_template(path):
    with open(path) as f:
        return Template(f.read())


def get_yaml(path):
    with open(path) as f:
        return yaml.load(f, yaml.Loader)


def write_template(path, template, d):
    with open(path, "w") as f:
        f.write(template.render(**d))

def get_meal_set(plan):
    return {m for d in plan.values() for m in d.values() if 'leftovers / ' not in m}


def get_meal_path(meal):
    return os.path.join(meals_path, f"{meal}.yaml")


def get_meal_data(meal_set):
    meal_data = {}
    for meal in meal_set:
        meal_path = get_meal_path(meal)
        meal_data[meal] = get_yaml(meal_path)
    return meal_data


def build_meals(meal_data):
    meal_template = get_template("./templates/meal.html")
    for meal, meal_datum in meal_data.items():
        meal_html_path = os.path.join(build_path, f"{meal}.html")
        write_template(meal_html_path, meal_template, meal_datum)

        
def build_week(plan, meal_data):
    plan_template = get_template("./templates/index.html")
    plan_html_path = os.path.join(build_path, "index.html")
    d = {"meal_data": meal_data, "plan": plan}
    write_template(plan_html_path, plan_template, d)        

    
def build():
    shutil.rmtree(build_path, ignore_errors=True)
    os.makedirs(build_path)
    shutil.copy("./style.css", build_path)
    plan = get_yaml(plan_path)
    meal_set = get_meal_set(plan)
    meal_data = get_meal_data(meal_set)
    build_meals(meal_data)
    build_week(plan, meal_data)

In [41]:
build()

{'mushroom-and-wild-rice-soup': {'name': 'Mushroom and wild rice soup', 'servings': 6, 'ingredients': ['60ml olive oil', '1 onion', '3 celery stalks', '1 green bell pepper', '10 garlic cloves', '1 tbsp miso paste', '2 tsp smoked paprika', '1 tsp sweet paprika', '1 tsp onion powder', '1 tsp garlic powder', '1 tsp cayenne', '500g dried red kidney beans', '3 dried bay leaves', '3 fresh thyme sprigs', '1 tsp soy sauce', '225ml wild rice'], 'steps': ['Turn on the sauté setting on the electric pressure cooker. Melt the butter, then add the onion. Cook, stirring occasionally, until translucent, about 5 minutes.', 'Add the mushrooms and 1 teaspoon salt, and cook, stirring occasionally, about 8 minutes. Add the celery, carrot, chopped garlic, thyme, garlic powder and several generous grinds of black pepper. Stir to combine.', 'Add the flour and stir until the vegetables are evenly coated and no white spots remain.', 'Pour in the stock and lemon juice, and turn off the heat.', 'Scrape the bottom

In [173]:
meal_data = {}
for meal_path in glob('./meals/*.yaml'):
    print(meal_path)
    meal_filename = os.path.split(meal_path)[1]
    meal_name = os.path.splitext(meal_filename)[0]
    meal_html_name = meal_name + '.html'
    meal_html_path = os.path.join(html_path, meal_html_name)
    meal_data[meal_name] = get_yaml(meal_path)
#     write_template(meal_html_path, meal_template, meal_data[meal_name])

./meals/breakfast-bars.yaml
./meals/breakfast-for-dinner.yaml
./meals/apple-pie-porridge.yaml
./meals/black-bean-soup.yaml
./meals/mushroom-and-wild-rice-soup.yaml
./meals/mac-and-cheese.yaml
./meals/ribollita.yaml
./meals/red-beans-and-rice.yaml


In [5]:
plan = get_yaml(plan_path)

In [12]:
meals

{'almond porridge',
 'banana pancake',
 'black-bean-soup',
 'fry-up',
 'mac-and-cheese',
 'mushroom-and-wild-rice-soup',
 'red-beans-and-rice',
 'ribollita',
 'sri-lankan-curry',
 'walnut porridge'}

In [174]:
shopping_list = defaultdict(float)
food_to_unit = dict()
for day in plan.values():
    for meal in day.values():
        for ingredient in meal_data[meal]['ingredients']:
            food, quantity, unit = parse_item(ingredient)
            if food in shopping_list:
                assert food_to_unit[food] == unit
            else:
                food_to_unit[food] = unit
            shopping_list[food] += quantity

In [175]:
for k in sorted(shopping_list):
    print(k, shopping_list[k])

aged balsamic vinegar 9.85784
avocado 0.5
bag of kale 1.0
baked beans 100.0
bananas 12.0
brown rice 300.0
bunch cilantro 1.0
butter 35.5164
can tomatoes 1.0
carrots 4.0
cayenne 9.85784
cayenne pepper 4.92892
celery stalks 13.0
cherries 360.0
cherry tomatoes 5.0
chili powder 71.0328
cranberries 360.0
cumin 35.5164
dessert apple 1.0
dried bay leaves 8.0
dried red kidney beans 1000.0
dried white beans 350.0
dry black beans 1000.0
eggs 2.0
fresh rosemary sprig 1.0
fresh thyme sprigs 9.0
garlic cloves 40.0
garlic powder 14.78676
green bell pepper 2.0
hashbrowns 2.0
hot pepper sauce 4.92892
jumbo oats 950.0
large leek 1.0
lemon 1.0
lime 2.0
macaroni 500.0
melted butter 360.0
milk 325.0
miso paste 35.5164
nuts (walnuts 20.0
olive oil 173.2746
onion 3.0
onion powder 9.85784
pecans 240.0
pepper 2.46446
pinch of cinnamon 1.0
pumpkin seeds 240.0
red onion 2.0
red pepper 2.0
red-pepper flakes 4.92892
salt 22.180139999999998
shredded cheddar cheese 500.0
shredded mozzarella cheese 250.0
shredded pa

In [59]:
shopping_list

defaultdict(float,
            {'jumbo oats': 150.0,
             'very ripe medium bananas': 2.0,
             'melted butter': 60.0,
             'cherries': 60.0,
             'cranberries': 60.0,
             'sunflower seeds': 40.0,
             'pumpkin seeds': 40.0,
             'walnuts': 40.0,
             'pecans': 40.0})

In [60]:
food_to_unit

{'jumbo oats': 'g',
 'very ripe medium bananas': None,
 'melted butter': 'g',
 'cherries': 'g',
 'cranberries': 'g',
 'sunflower seeds': 'g',
 'pumpkin seeds': 'g',
 'walnuts': 'g',
 'pecans': 'g'}

In [55]:
quantity

40.0

In [56]:
unit

'g'

In [1]:
import re
from collections import defaultdict

def parse_item(item):
    p = re.compile(r'(?P<quantity>[0-9\.]+)\s*(?P<unit>ml|g|tbsp|tsp|L)?\s+(?P<food>.+)')
    conversion = {
        'tbsp': 17.7582,
        'tsp': 4.92892,
        'L': 1000,
    }
    m = p.match(item)
    gd = m.groupdict()
    quantity = float(gd['quantity'])
    food = gd['food']
    unit = gd['unit'] if 'unit' in gd else None
    if gd['unit']:
        unit = gd['unit']
        if unit in conversion:
            quantity *= conversion[unit]
            unit = 'ml'
    food = re.sub(r',.*$', '', food).strip()
    food = re.sub(r'\(.*\)', '', food).strip()
    return food, quantity, unit

In [2]:
parse_item('1 tbsp nuts (nice)')

('nuts', 17.7582, 'ml')

In [118]:
f = _[0]

'nuts'

In [ ]:
re.sub(r'[^\w]', ' ', s)

In [26]:
plan_html_path = os.path.join(html_path, 'index.html')
d = {'meal_data':meal_data, 'plan': plan}
write_template(plan_html_path, plan_template, d)

In [27]:
plan

{'monday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'tuesday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'wednesday': {'breakfast': 'breakfast-bars',
  'dinner': 'breakfast-for-dinner'},
 'thursday': {'breakfast': 'breakfast-bars', 'dinner': 'red-beans-and-rice'},
 'friday': {'breakfast': 'apple-pie-porridge',
  'dinner': 'mushroom-and-wild-rice-soup'},
 'saturday': {'breakfast': 'breakfast-bars', 'dinner': 'mac-and-cheese'},
 'sunday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'}}

In [28]:
meal_data['breakfast-bars']['name']

'Breakfast bars'

In [3]:
plan

{'monday': {'breakfast': 'breakfast-bars', 'dinner': 'ribollita'},
 'tuesday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'wednesday': {'breakfast': 'breakfast-bars',
  'dinner': 'breakfast-for-dinner'},
 'thursday': {'breakfast': 'breakfast-bars', 'dinner': 'red-beans-and-rice'},
 'friday': {'breakfast': 'apple-pie-porridge',
  'dinner': 'mushroom-and-wild-rice-soup'},
 'saturday': {'breakfast': 'breakfast-bars', 'dinner': 'mac-and-cheese'},
 'sunday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'}}